# Artifice

Notebook for running GLUE tasks.

# Setup

## Modules

In [1]:
import random
import csv
import pathlib
import itertools
from datetime import datetime
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

from utils.model import Model
from utils.cls import extract_cls_embeddings
from utils.mean_pooling import mean_pooling

In [2]:
# standardized default seed
seed = 7
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

## Device

In [3]:
device_name = "cpu"  # default device is CPU
if torch.cuda.is_available():
    device_name = "cuda"  # CUDA for NVIDIA GPU
elif torch.backends.mps.is_available():
    device_name = torch.device("mps")  # Metal Performance Shaders for Apple M-series GPU
device = torch.device(device_name)
print(device_name)

mps


## User parameters

# Training loop

In [5]:
param_grid = {
    'num_epochs': [50],
    'batch_size': [32, 512],
    'learning_rate': [1e-2, 1e-3],
    'category': ["BC"],
    'weight_decay':[1e-2, 1e-4],
    'patience': [3],
    'min_delta': [0],
    'device': [device_name]
}

# Create a list of all combinations of hyperparameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
print(f"{len(all_params)} hyperparameter combinations")

8 hyperparameter combinations


In [11]:
# setup for logging
console_output_filename = f'./output/console_output.txt'
with open(console_output_filename, 'w') as logfile:
    logfile.write('\n\nBEGIN TRAINING LOOP\n\n')
# setup for saving results
results_folder = pathlib.Path(f"results/")
results_folder.mkdir(parents=True, exist_ok=True)
save_file_id = datetime.now().strftime("%Y%m%d_%H%M%S")
results_file = results_folder / f"val_{save_file_id}.csv"

with open(results_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    headers = list(all_params[0].keys())
    writer.writerow(['mcc', 'f1', 'accuracy'] + headers)
print(f"saving results to ./{results_file}")
# saves best accuracy for progress bar display
best_acc = 0.0
    
# Iterate over all combinations of hyperparameters
bar = tqdm(enumerate(all_params), total=len(all_params))
for i, params in bar:
    # formatting params to display
    print_params = params.copy()
    for param in ['category', 'device']:
        del print_params[param]
    
    # Initialize the model with current set of hyperparameters
    feed_forward = Model(**params)

    metrics = feed_forward.fit(X_train, Y_train, X_val, Y_val)

    epoch, val_loss, val_accuracy, val_f1, val_mcc = metrics["epoch"], metrics["loss"], metrics["acc"], metrics["f1"], metrics["mcc"]
    best_acc = max(best_acc, val_accuracy)
    bar.set_description(f"Best Acc: {best_acc:.5f}, Last test: {val_accuracy:.5f}")

    # Write stats to log file
    with open(console_output_filename, 'a') as logfile:
        logfile.write(f"\n\nTraining with parameters:\n{print_params}")
        logfile.write(f"\nEarly stopped on epoch: {epoch}")
        logfile.write(f"\nValidation accuracy: {val_accuracy}")
        logfile.write(f"\nValidation f1-score: {val_f1}")
        logfile.write(f"\nValidation MCC     : {val_mcc}")
    # write to results csv
    with open(results_file, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([val_mcc, val_f1, val_accuracy] + list(params.values()))

saving results to ./results/val_20231202_142046.csv


  0%|          | 0/8 [00:00<?, ?it/s]

NameError: name 'X_train' is not defined

In [ ]:
results_df = pd.read_csv(results_file)
# results_df = pd.read_csv("output/val_results_cola_20231127_151717.csv")
if task_config.class_type in ["BC", "MC"]:
    metric = "accuracy"
    best = results_df[metric].max()
    best_row = results_df[results_df[metric] == best]
elif task_config.class_type == "R":
    metric = "pearson"
    best = results_df[metric].max()
    best_row = results_df[results_df[metric] == best]
    
print(f"Best {metric}: {best:.5f}")
best_row